In [10]:
# pip install tabulate 해야함 !!
# tabulate 라이브러리는 ide에서 사용할 떄 적용
# 주피터나 코랩에서는 그냥 print하면 됨
import pandas as pd
from tabulate import tabulate
import re


# tabulate 라이브러리 사용해서 로컬에서 print 대신 print_df 함수 사용해서 Jupyter notebook 처럼 보기
def print_df(d):
    print(tabulate(d, headers='keys', tablefmt='psql'))


def date_preprocess(d):
    d['date'] = d['date'].apply(lambda x: re.sub('[./]', '', x))
    return d


finance_news = pd.read_excel('naver_finance_news.xlsx', usecols=[2, 0])
finance_news = date_preprocess(finance_news)

naver_news = pd.read_excel('naver_news.xlsx')
naver_news = date_preprocess(naver_news)

report = pd.read_excel('report.xlsx')
report = date_preprocess(report)


df = naver_news.append(finance_news, ignore_index=True)
df = df.append(report, ignore_index=True)
df = df.sort_values('date', ignore_index=True)


# 날짜 중복삭제 & title 합치기
def eliminate_dup_date(d, new_d):
    for date, tit, dup in d.values:
        if not dup:
            new_d = new_d.append(pd.Series([date, tit], index=new_d.columns), ignore_index=True)
        else:
            new_d.iloc[-1, -1] = new_d.iloc[-1, -1] + ', ' + tit
    return new_d


# df에 중복 여부 컬럼 추가
df['duplicated'] = df.duplicated(['date'])
print(df.tail(10))

res = eliminate_dup_date(df, pd.DataFrame(columns=['date', 'title']))
print(res.tail(10))

# date 열 datetime 형식으로 변경하고 인덱스 지정
res['date'] = pd.to_datetime(res['date'], format='%Y%m%d').dt.normalize()
res = res.set_index('date')
print(res.tail(10))

# 중복 제거 전후 데이터셋의 크기 비교
print(f'중복 데이터셋의 크기: {len(df)}, 중복 제거한 데이터셋의 크기: {len(res)}')
print(df.date.unique())

           date                                       title  duplicated
16993  20220601                 '삼성효과' 걷어내니…더 돋보인 손해보험사의 진격        True
16994  20220601  삼성·SK 공장 있는 곳이라…경기지사 선거도 뜨거운 ‘반도체 전쟁’ [...        True
16995  20220601  삼성전자 주가 대체 어디로 "경영진도 근심 가득"…6만원까지 또 미끄러...        True
16996  20220601             삼성 세탁기·건조기 신제품 동시구매시 최대 20만원 할인        True
16997  20220601     "선곡 미쳤네" 누리꾼 극찬한 삼성 세탁기 광고 음악…누가 불렀나 보니        True
16998  20220601            이재용 삼성전자 부회장 주식평가액 올 들어 1.1조원 증발        True
16999  20220601            올들어 이재용 삼성전자 부회장 주식평가액 1.1조원 줄었다        True
17000  20220601      "삼성·LG 제쳤다"…1분기 노트북 시장 장악한 이 기업, 어디길래?        True
17001  20220601          '갤럭시 Z4' 기대감 커지는데…삼성 마냥 웃을 수 없는 이유        True
17002  20220601      [베스트 애널리스트 추천 종목]삼성전자, 전략 자산의 중심 ‘반도체’        True
         date                                              title
279  20220523  삼성전자, 워커힐과 함께 ‘시네마 스위트 with 삼성 더 프리미어’ ..., 라온...
280  20220524  [속보] 삼성, 반도체 등 핵심인력 확보…5년간 8만명 채용, 삼성, 5년간 8만명...
281  20220525